# Multivariate Linear Regression

Multivariate linear regression is used when we have more than one feature x, where: <br>
$ \hat{y} = w_{1}*x_{1} + w_{2}*x_{2} + ... + w_{n}*x_{n} + b $

in this situation w and x are vectors $ \vec{w} $, resp. $ \vec{x} $, so we can rewrite the formula: <br>
Without vectorization: <br>
$ f_{\vec{w}, b}(\vec{x}) = \sum_{j=1}^{n} w_{j}*x_{j} + b \, $   ,where n is number of features <br>
Or in vector format: <br>
$ f_{\vec{w}, b}(\vec{x}) = \vec{w} \cdot \vec{x} + b $   ,where $ \cdot $ is a dot product of two vectors <br>

Note: vectorization is not necessary and we can deal with calculations using loops. The big pros of vectorization are that it is much faster than loops.


## Cost function
for cost function and gradient descent the formulas a derived from univariate with analogy to the multivariate formula above <br>
$ J(\vec{w}, b) = \frac{1}{2*m}*\sum_{i=1}^{m}(f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)})^{2} \, $

## Gradient descent
Now we have $ \vec{w} $, that means we have to update all $ w_{1} ... w_{n} $ <br>
<br>
$ w_{1} = w_{1} - \alpha * \frac{\partial}{\partial w_{1}} J(\vec{w}, b) $ ,where we have partial derivative by $ w_{1} $ <br>
$ w_{1} = w_{1} - \alpha * \frac{1}{m}*\sum_{i=1}^{m}(f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)})*x_{1}^{(i)} \, $ <br>
<br>
b is a number, therefore: <br>
$ b = b - \alpha * \frac{1}{m}*\sum_{i=1}^{m}(f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)}) $

## Implementation of gradient descent for multivariate linear regression

First let's load data, in this example, I will use a housing data set, which I will reduce to only 3 features and 50 records

In [1]:
import pandas as pd

df = pd.read_csv('./data/Housing.csv')
print(df.shape)
print(df.head())

(545, 13)
      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [16]:
# let's keep just the target column (price) and area, bedrooms, and bathrooms
df = df[['price', 'area', 'bedrooms', 'bathrooms']]
df = df.sample(n=50) #for time simplicity we will choose only 50 rows

print(df.describe())

              price         area  bedrooms  bathrooms
count  5.000000e+01    50.000000  50.00000  50.000000
mean   4.409230e+06  4485.560000   2.82000   1.240000
std    2.132772e+06  1721.511298   0.66055   0.431419
min    1.890000e+06  1836.000000   2.00000   1.000000
25%    3.080000e+06  3292.500000   2.00000   1.000000
50%    3.902500e+06  4140.000000   3.00000   1.000000
75%    4.838750e+06  5421.250000   3.00000   1.000000
max    1.221500e+07  9166.000000   4.00000   2.000000


As we can see we have very different scales for each feature. The area comes from 1836 to 9166 m2 but bedrooms are just from 2 to 4. In the case when features are very different scale gradient can easily overshoot and it is recommended to perform feature normalization. There are two common feature normalizations:
1) Z-score: $ x_{n} = \frac{x_{n} - \mu_{n}}{\sigma_{n}} $ <br>
2) MinMax : $ x_{n} = \frac{x_{n} - \mu_{n}}{max(x_{n}) - min(x_{n})} $

We will continue with Z-score

In [22]:
features = ['area', 'bedrooms', 'bathrooms']
new_features = []
for feature in features:
    df[feature + '_s'] = (df[feature] - df[feature].mean()) / df[feature].std()
    new_features.append(feature + '_s')

df.head()

,price,area,bedrooms,bathrooms,area_s,bedrooms_s,bathrooms_s
462,3080000,2160,3,1,-1.350883,0.272500,-0.556304
474,2975000,4352,4,1,-0.077583,1.786390,-0.556304
405,3465000,3060,3,1,-0.828086,0.272500,-0.556304
521,2408000,3635,2,1,-0.494078,-1.241389,-0.556304
214,4865000,4350,2,1,-0.078745,-1.241389,-0.556304


In [46]:
import numpy as np
#split on features (X) and target (y)
X = df[new_features]
y = df['price']

print(X.iloc[1].to_numpy())
print(y.iloc[0])

[-0.07758299  1.78638966 -0.55630359]
3080000


In [53]:
#calculation of gradient (a derivative of cost function J(w,b))
def calc_gradient(x, y, w, b):
    m, n = x.shape
    de_w = np.zeros((n,))
    de_b = 0
    
    for i in range(m):
        x_i = X.iloc[i].to_numpy()  #convert pandas row to numpy vector
        err = np.dot(x_i, w) + b - y.iloc[i]  #calculate error
        for j in range(n):
            de_w[j] += err*x_i[j]     #update weights
        de_b += err     # update bias

    de_w = de_w / m
    de_b = de_b / m
    
    return de_w, de_b

#updating w, b value to the local minima
def gradient_descent(x, y, w_in, b_in, alpha, num_iter):
    for i in range(num_iter):
        de_w, de_b = calc_gradient(x, y, w_in, b_in)  # get gradient
        w_in = w_in - alpha * de_w
        b_in = b_in - alpha * de_b

        if i % 1000 == 0:
            print(f'Iteration {i:4} | w : {w_in} | b : {b_in: 8.2f}')
    return w_in, b_in

In [54]:
# initialize parameters
w_init = np.zeros((X.shape[1],))
b_init = 0

num_iters = 10000
alpha = 1.0e-2

w_fin, b_fin = gradient_descent(X, y, w_init, b_init, alpha, num_iters)
print(f'(w,b) found by gradient descent: ({w_fin}, {b_fin:8.4f})')

Iteration    0 | w : [13904.55226104  6548.04764115 12919.79855138] | b :  44092.30
Iteration 1000 | w : [1125702.76797782  207819.00591714  905944.12633548] | b :  4409041.55
Iteration 2000 | w : [1125612.01387661  207496.83086716  906284.69040665] | b :  4409229.99
Iteration 3000 | w : [1125611.41429131  207495.9314091   906285.89493106] | b :  4409230.00
Iteration 4000 | w : [1125611.41216074  207495.92831336  906285.8991261 ] | b :  4409230.00
Iteration 5000 | w : [1125611.41215332  207495.92830259  906285.8991407 ] | b :  4409230.00
Iteration 6000 | w : [1125611.41215331  207495.92830256  906285.89914074] | b :  4409230.00
Iteration 7000 | w : [1125611.41215331  207495.92830256  906285.89914074] | b :  4409230.00
Iteration 8000 | w : [1125611.41215331  207495.92830256  906285.89914074] | b :  4409230.00
Iteration 9000 | w : [1125611.41215331  207495.92830256  906285.89914074] | b :  4409230.00
(w,b) found by gradient descent: ([1125611.41215331  207495.92830256  906285.89914074], 

#### We have found optimal weights ($\vec{w}$) and bias (b)
### Let's compare results with sci-kit learn library

In [69]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

lr_model.fit(X, y)
y_pred = lr_model.predict(X)

sk_w = lr_model.coef_
sk_b = lr_model.intercept_
np.set_printoptions(precision=4)  #numpy print only 4 decimal places
w_diff = (w_fin - sk_w).round(decimals=4)
print(f'w => manually: {w_fin}  |  sklearn  {sk_w}   |  diff: {w_diff}')
print(f'b => manually: {b_fin:8.4f}  |  sklearn  {sk_b:8.4f}   |  diff: {b_fin - sk_b:8.4f}')

w => manually: [1125611.4122  207495.9283  906285.8991]  |  sklearn  [1125611.4122  207495.9283  906285.8991]   |  diff: [ 0.  0. -0.]
b => manually: 4409230.0000  |  sklearn  4409230.0000   |  diff:  -0.0000
